In [34]:
# The code was removed by Watson Studio for sharing.

In [1]:
#Data Extracted from The COVID Tracking Project (as of 4/8/2020)
#https://covidtracking.com/
import pandas as pd
import requests
import json
import datetime
!pip install pycountry
import pycountry

     |████████████████████████████████| 10.0MB 7.3MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/a2/98/bf/f0fa1c6bf8cf2cbdb750d583f84be51c2cd8272460b8b36bd3
Successfully built pycountry


In [2]:
raw_response = requests.get("https://covidtracking.com/api/states/daily").text
raw_data = pd.DataFrame.from_dict(json.loads(raw_response))

In [3]:
data = raw_data.fillna(0)
data['Date'] = pd.to_datetime(data['date'].astype(str), format='%Y%m%d')
data = data.rename(
    columns={
        "state": "ISO3166-2",
        "positive": "Positive",
        "negative": "Negative",
        "pending": "Pending",
        "death": "Death",
        "totalTestResults": "Total",
        "hospitalized": "Hospitalized"
    })
data = data.drop(labels=['dateChecked', "date"], axis='columns')
data['Country/Region'] = "United States"
data['ISO3166-1'] = "US"

In [4]:
states = {k.code.replace("US-", ""): k.name for k in pycountry.subdivisions.get(country_code="US")}

In [5]:
data["Province/State"] = data["ISO3166-2"].apply(lambda x: states[x])

In [6]:
sorted_data = data.sort_values(by=['Province/State'] + ['Date'], ascending=True)

In [18]:
sorted_data['Current_Day_Positive_Cases'] = sorted_data['Positive'] - sorted_data.groupby(['Province/State'])["Positive"].shift(1, fill_value=0)
sorted_data['Current_Day_Total_Tests'] = sorted_data['Total'] - sorted_data.groupby(['Province/State'])["Total"].shift(1, fill_value=0)
sorted_data['Current_Day_Negative_Cases'] = sorted_data['Negative'] - sorted_data.groupby(['Province/State'])["Negative"].shift(1, fill_value=0)
sorted_data['Current_Day_Pending_Cases'] = sorted_data['Pending'] - sorted_data.groupby(['Province/State'])["Pending"].shift(1, fill_value=0)
sorted_data['Current_Day_Deaths'] = sorted_data['Death'] - sorted_data.groupby(['Province/State'])["Death"].shift(1, fill_value=0)
sorted_data['Current_Day_Hospitalized'] = sorted_data['Hospitalized'] - sorted_data.groupby(['Province/State'])["Hospitalized"].shift(1, fill_value=0)
sorted_data['Prior_Day_Positive_Cases'] = sorted_data.groupby('Province/State')['Positive'].shift(1, fill_value=0)
sorted_data['Prior_Day_Deaths'] = sorted_data.groupby('Province/State')['Death'].shift(1, fill_value=0)
sorted_data['Prior_Day_Total_Tests'] = sorted_data.groupby('Province/State')['Total'].shift(1, fill_value=0)
sorted_data['Pct_Change_Positive_Cases'] = (sorted_data['Positive'] - sorted_data['Prior_Day_Positive_Cases'])/sorted_data['Prior_Day_Positive_Cases']
sorted_data['Pct_Change_Deaths'] = (sorted_data['Death'] - sorted_data['Prior_Day_Deaths'])/sorted_data['Prior_Day_Deaths']
sorted_data['Pct_Change_Total_Tests'] = (sorted_data['Total'] - sorted_data['Prior_Day_Total_Tests'])/sorted_data['Prior_Day_Total_Tests']
sorted_data['Cumulative_Positive_Cases'] = sorted_data['Positive']
sorted_data['Cumulative_Total_Tests'] = sorted_data['Total']
sorted_data['Cumulative_Deaths'] = sorted_data['Death']

In [19]:
rearranged_data = sorted_data.filter(items=['Country/Region', 'Province/State', 'Date',
                               'Cumulative_Positive_Cases', 'Current_Day_Positive_Cases',
                               'Prior_Day_Positive_Cases','Pct_Change_Positive_Cases',
                               'Cumulative_Total_Tests', 'Current_Day_Total_Tests',
                               'Prior_Day_Total_Tests', 'Pct_Change_Total_Tests',
                               'Cumulative_Deaths', 'Current_Day_Deaths',
                               'Prior_Day_Deaths', 'Pct_Change_Deaths',
                               'ISO3166-1', 'ISO3166-2'])

In [21]:
rearranged_data.loc[:, "Last_Update_Date"] = datetime.datetime.utcnow()
rearranged_data.head(100)

,Country/Region,Province/State,Date,Cumulative_Positive_Cases,Current_Day_Positive_Cases,Prior_Day_Positive_Cases,Pct_Change_Positive_Cases,Cumulative_Total_Tests,Current_Day_Total_Tests,Prior_Day_Total_Tests,Pct_Change_Total_Tests,Cumulative_Deaths,Current_Day_Deaths,Prior_Day_Deaths,Pct_Change_Deaths,ISO3166-1,ISO3166-2,Last_Update_Date
1697,United States,Alabama,2020-03-07,0.0,0.0,0.0,NaN,0,0,0,NaN,0.0,0.0,0.0,NaN,US,AL,2020-04-08 18:18:33.328869
1646,United States,Alabama,2020-03-08,0.0,0.0,0.0,NaN,0,0,0,NaN,0.0,0.0,0.0,NaN,US,AL,2020-04-08 18:18:33.328869
1595,United States,Alabama,2020-03-09,0.0,0.0,0.0,NaN,0,0,0,NaN,0.0,0.0,0.0,NaN,US,AL,2020-04-08 18:18:33.328869
1544,United States,Alabama,2020-03-10,0.0,0.0,0.0,NaN,0,0,0,NaN,0.0,0.0,0.0,NaN,US,AL,2020-04-08 18:18:33.328869
1493,United States,Alabama,2020-03-11,0.0,0.0,0.0,NaN,10,10,0,inf,0.0,0.0,0.0,NaN,US,AL,2020-04-08 18:18:33.328869
1442,United States,Alabama,2020-03-12,0.0,0.0,0.0,NaN,10,0,10,0.000000,0.0,0.0,0.0,NaN,US,AL,2020-04-08 18:18:33.328869
1391,United States,Alabama,2020-03-13,1.0,1.0,0.0,inf,12,2,10,0.200000,0.0,0.0,0.0,NaN,US,AL,2020-04-08 18:18:33.328869
1340,United States,Alabama,2020-03-14,6.0,5.0,1.0,5.000000,28,16,12,1.333333,0.0,0.0,0.0,NaN,US,AL,2020-04-08 18:18:33.328869
1289,United States,Alabama,2020-03-15,12.0,6.0,6.0,1.000000,40,12,28,0.428571,0.0,0.0,0.0,NaN,US,AL,2020-04-08 18:18:33.328869
1233,United States,Alabama,2020-03-16,28.0,16.0,12.0,1.333333,56,16,40,0.400000,0.0,0.0,0.0,NaN,US,AL,2020-04-08 18:18:33.328869


In [22]:
MA_only = rearranged_data[(rearranged_data['Province/State']=='Massachusetts')]
MA_only.head()

,Country/Region,Province/State,Date,Cumulative_Positive_Cases,Current_Day_Positive_Cases,Prior_Day_Positive_Cases,Pct_Change_Positive_Cases,Cumulative_Total_Tests,Current_Day_Total_Tests,Prior_Day_Total_Tests,Pct_Change_Total_Tests,Cumulative_Deaths,Current_Day_Deaths,Prior_Day_Deaths,Pct_Change_Deaths,ISO3166-1,ISO3166-2,Last_Update_Date
1812,United States,Massachusetts,2020-03-04,2.0,2.0,0.0,inf,2,2,0,inf,0.0,0.0,0.0,NaN,US,MA,2020-04-08 18:18:33.328869
1790,United States,Massachusetts,2020-03-05,2.0,0.0,2.0,0.000,2,0,2,0.000,0.0,0.0,0.0,NaN,US,MA,2020-04-08 18:18:33.328869
1761,United States,Massachusetts,2020-03-06,8.0,6.0,2.0,3.000,8,6,2,3.000,0.0,0.0,0.0,NaN,US,MA,2020-04-08 18:18:33.328869
1715,United States,Massachusetts,2020-03-07,13.0,5.0,8.0,0.625,13,5,8,0.625,0.0,0.0,0.0,NaN,US,MA,2020-04-08 18:18:33.328869
1664,United States,Massachusetts,2020-03-08,13.0,0.0,13.0,0.000,13,0,13,0.000,0.0,0.0,0.0,NaN,US,MA,2020-04-08 18:18:33.328869


In [26]:
MA_only['Cumulative_Mortality_Rate']=MA_only['Cumulative_Deaths']/MA_only['Cumulative_Positive_Cases']
MA_only['Cumulative_Positive_Test_Rate'] = MA_only['Cumulative_Positive_Cases']/MA_only['Cumulative_Total_Tests']
MA_only['Current_Day_Mortality_Rate']=MA_only['Current_Day_Deaths']/MA_only['Current_Day_Positive_Cases']
MA_only['Current_Day_Positive_Test_Rate'] = MA_only['Current_Day_Positive_Cases']/MA_only['Current_Day_Total_Tests']
MA_only.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

,Country/Region,Province/State,Date,Cumulative_Positive_Cases,Current_Day_Positive_Cases,Prior_Day_Positive_Cases,Pct_Change_Positive_Cases,Cumulative_Total_Tests,Current_Day_Total_Tests,Prior_Day_Total_Tests,...,Current_Day_Deaths,Prior_Day_Deaths,Pct_Change_Deaths,ISO3166-1,ISO3166-2,Last_Update_Date,Cumulative_Mortality_Rate,Cumulative_Positive_Test_Rate,Current_Day_Mortality_Rate,Current_Day_Positive_Test_Rate
1812,United States,Massachusetts,2020-03-04,2.0,2.0,0.0,inf,2,2,0,...,0.0,0.0,NaN,US,MA,2020-04-08 18:18:33.328869,0.0,1.0,0.0,1.0
1790,United States,Massachusetts,2020-03-05,2.0,0.0,2.0,0.000,2,0,2,...,0.0,0.0,NaN,US,MA,2020-04-08 18:18:33.328869,0.0,1.0,NaN,NaN
1761,United States,Massachusetts,2020-03-06,8.0,6.0,2.0,3.000,8,6,2,...,0.0,0.0,NaN,US,MA,2020-04-08 18:18:33.328869,0.0,1.0,0.0,1.0
1715,United States,Massachusetts,2020-03-07,13.0,5.0,8.0,0.625,13,5,8,...,0.0,0.0,NaN,US,MA,2020-04-08 18:18:33.328869,0.0,1.0,0.0,1.0
1664,United States,Massachusetts,2020-03-08,13.0,0.0,13.0,0.000,13,0,13,...,0.0,0.0,NaN,US,MA,2020-04-08 18:18:33.328869,0.0,1.0,NaN,NaN


In [31]:
MA_only['MA7_Pct_Change_Positive_Cases'] = MA_only.rolling(window=7)['Pct_Change_Positive_Cases'].mean()
MA_only['MA7_Pct_Change_Total_Tests'] = MA_only.rolling(window=7)['Pct_Change_Total_Tests'].mean()
MA_only['MA7_Pct_Change_Deaths'] = MA_only.rolling(window=7)['Pct_Change_Deaths'].mean()
MA_only['MA7_Cumulative_Mortality_Rate'] = MA_only.rolling(window=7)['Cumulative_Mortality_Rate'].mean()
MA_only['MA7_Current_Day_Mortality_Rate'] = MA_only.rolling(window=7)['Current_Day_Mortality_Rate'].mean()
MA_only['MA7_Cumulative_Positive_Test_Rate'] = MA_only.rolling(window=7)['Cumulative_Positive_Test_Rate'].mean()
MA_only['MA7_Current_Day_Positive_Test_Rate'] = MA_only.rolling(window=7)['Current_Day_Positive_Test_Rate'].mean()
MA_only.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

,Country/Region,Province/State,Date,Cumulative_Positive_Cases,Current_Day_Positive_Cases,Prior_Day_Positive_Cases,Pct_Change_Positive_Cases,Cumulative_Total_Tests,Current_Day_Total_Tests,Prior_Day_Total_Tests,...,Cumulative_Positive_Test_Rate,Current_Day_Mortality_Rate,Current_Day_Positive_Test_Rate,MA7_Pct_Change_Positive_Cases,MA7_Pct_Change_Total_Tests,MA7_Pct_Change_Deaths,MA7_Cumulative_Mortality_Rate,MA7_Current_Day_Mortality_Rate,MA7_Cumulative_Positive_Test_Rate,MA7_Current_Day_Positive_Test_Rate
1812,United States,Massachusetts,2020-03-04,2.0,2.0,0.0,inf,2,2,0,...,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1790,United States,Massachusetts,2020-03-05,2.0,0.0,2.0,0.000,2,0,2,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1761,United States,Massachusetts,2020-03-06,8.0,6.0,2.0,3.000,8,6,2,...,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1715,United States,Massachusetts,2020-03-07,13.0,5.0,8.0,0.625,13,5,8,...,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1664,United States,Massachusetts,2020-03-08,13.0,0.0,13.0,0.000,13,0,13,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
#from project_lib import Project
project = Project("covid19-donotdelete-pr-ysdyv5czxoifrj","ef868927-b989-4aa5-b2ad-753b69d00eb5","p-a323bb79303649f1b354d3416d15f0b502d52507")
df = MA_only
df
project.save_data(file_name = "MA_COVID.csv",data = df.to_csv(index=False))

{'file_name': 'MA_COVID.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'covid19-donotdelete-pr-ysdyv5czxoifrj',
 'asset_id': 'e972e749-605b-4116-9a80-1f49cad3df4e'}